In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
base_model=MobileNet(weights='imagenet',include_top=False)

In [ ]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(3,activation='softmax')(x) 

In [ ]:
model=Model(inputs=base_model.input,outputs=preds)

In [ ]:
for layer in model.layers:
    layer.trainable=False
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_generator=train_datagen.flow_from_directory('/storage/Chest XRay/',
                                                 target_size=(150,150),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,steps_per_epoch=step_size_train,epochs=10)

In [ ]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(150,150))
    img_tensor = image.img_to_array(img)                    
    img_tensor = np.expand_dims(img_tensor, axis=0)         
    img_tensor /= 255.                                     

    if show:
        plt.imshow(img_tensor[0])
        plt.axis('off')
        plt.show()

    return img_tensor


In [ ]:
img_path = '/storage/Chest XRay Test/Normal/IM-0001-0001.jpeg' 
new_image = load_image(img_path)
pred = model.predict(new_image)
print(pred)

In [ ]:
img_path = '/storage/Chest XRay Test/PNEUMONIA/person3_virus_16.jpeg' 
new_image = load_image(img_path)
pred = model.predict(new_image)
print(pred)

In [ ]:
img_path = '/storage/Chest XRay Test/Normal/IM-0010-0001.jpeg' 
new_image = load_image(img_path)
pred = model.predict(new_image)
print(pred)